In [ ]:
#Script to read data from input transcript file for all seasons
#Extracted data will be stored into a file ../dataset/friends_dataset_allseasons_tmp.csv
import re
import sys 
import os
import pandas as pd

#key words to skip
keywords=['End','Closing Credits','1017-1018 - The Last One','OPENING CREDITS','COMMERCIAL BREAK','THE END','Opening Credits','Commercial break','END','Credits','OPENING TITLES','CLOSING CREDITS']
keywordstr = '^\s*(END|Closing Credits|OPENING CREDITS|COMMERCIAL(\s+BREAK)?|CLOSING TITLES|THE END|OPENING TITLES|Commercial|Credits|HTMLed by|To Be Continued\.*|The Next Morning)\s*$'
# Creating an empty Dataframe with column names only
df = pd.DataFrame(columns=['Season', 'Episode', 'Episode Name','Location','Character','Dialogue'])


#Reading all transcript files from given path

def readfiles(dirpath):
    dff = pd.DataFrame(columns=['File', 'Season', 'Episode'])
    for file in os.listdir(dirpath):
        if file.endswith(".csv"):
            fileName=os.path.join(dirpath, file)
            if re.search("^\d{3}\.",file):
                mo = re.search("^(\d)(\d\d)\.",file)
                season = mo.group(1)
                episode = mo.group(2)
            elif re.search("^\d{4}\.",file):
                mo = re.search("^(\d\d)(\d\d)\.",file)
                season = mo.group(1)
                episode = mo.group(2)
            elif re.search("^\d{3}-\d{3}\.",file):
                mo = re.search("^(\d)(\d\d)-\d(\d\d)\.",file)
                season = mo.group(1)
                episode = mo.group(2) + "-" + mo.group(3)            
            elif re.search("^\d{4}-\d{4}\.",file):
                mo = re.search("^(\d\d)(\d\d)-\d\d(\d\d)\.",file)
                season = mo.group(1)
                episode = mo.group(2) + "-" + mo.group(3)
            else:
                print("Error - File name is not in correct format: ", filename)
            dff = dff.append({'File': fileName,'Season':season, 'Episode':episode}, ignore_index=True)
            #print ("File: {}.Season: {}, Episode: {}".format(fileName,season,episode))
    dff["Season"] = dff['Season'].astype('int')
    dff=dff.sort_values(['Season', 'Episode'], ascending=[True, True])
    return dff

dfn = readfiles('../transcripts/allseasons')
#print (dfn)

# iterate through each row and select  
for index, row in dfn.iterrows(): 
    print (row["File"], row["Season"],row['Episode'])
    fileName=row["File"]
    season=row["Season"]
    episode=row['Episode']

    #open input transcript file
    #mbcs
    with open(fileName,encoding="utf-8") as tsf: 
    #with open(fileName) as tsf: 
        #Lines = tsf.readlines()
        data = tsf.read()
        #print (data)
        #(?<!^) --> Not starting with string
        #print ("--------------------------------------------------------------------------------------")
        data_new = re.sub("([^\n])((\[|\()\s*(Flashback |Tag )?scene(:|;|,| ))",r"\1\n\2",data,flags=re.I)
        #print (data_new)
        Lines = list(data_new.split("\n")) 
        episodeName=""
        episodeFlag = 1
        sceneName = ""
        locationName = ""
        for linedata in Lines:
            characterName = ""
            dialogue = ""
            #print ("--{}--".format(line))
            line = linedata.strip('\r?\n')
            if re.match("^\s*$",line):
                continue
            if episodeFlag == 1 and not re.search("^\s*(\[|\()\s*(Flashback |Tag )?scene(:|;|,| )",line,re.IGNORECASE):
                episodeName = re.sub(r"^\s+|\s+$","",line,flags=re.I)
                episodeFlag=0
                #print ("Season: {} and Episode: {}".format(season,episodeName))
                continue
            
            if not re.match("^\s*(Dutch Phrases by|Produced by|Flashback clips transcribed by|With Minot Adjustments by|Transcript by|Episodes Orginally Transcribed by|With Help From|Episodes Ori?ginally Transcribed by|With Scenes Taken From Episodes Transcribed by|TIME LAPSE|Aired|Directed by|Ending Credits|Parts? |Commerical Break\.?|Story by|Teleplay by|Minor additions and adjustments by|Contrl\d+\.gif|Originally written|Tran?scribed by|Written by|Additional transcribing by|With Minor Adjustments by|\(Note: The previously unseen).*$", line,re.IGNORECASE):
                if re.search("^\s*(\[|\()\s*(Russian to Roman alphabet|Flashback |Tag )?scene(:|;|,| )",line,re.IGNORECASE):
                    mo = re.search("^\s*(\[|\()\s*(Flashback |Tag )?scene(:|;|,| )(.+?)$",line,re.IGNORECASE)
                    sceneData = mo.group(4)
                    if re.search("continued from earlier",sceneData,re.IGNORECASE):
                        continue
                    elif not re.match("^\s*$",sceneData):
                        smo = re.search("^(....+?)([,;:\.\-\]]|$)",sceneData,re.IGNORECASE)
                        
                        #print ("Scene Data: ", sceneData )
                        try:
                            locationName = re.sub(r"^\s+|\s+$","",smo.group(1),flags=re.I).lower()                            
                        except:
                            print("Something went wrong when extracting Scene Details: ", sceneData)
                            sys.exit()
                          
                        if re.match("^\s*$",locationName):
                            print ("Error ... Location Name is not available",sceneData)
                            sys.exit()
                        else:
                            continue
                    else:
                        print ("Error ... Scene Details is not available")
                        sys.exit()
                elif re.search("^\s*(\*|<|\[|\(|\{)",line,re.IGNORECASE):
                    continue
                elif re.match("^\s*Commercial Break\s*$",line):
                    continue
                elif re.search("^([^:]+)\s*:",line,re.IGNORECASE):
                    dmo = re.search("^([^:]+)\s*:\s*(.*)$",line,re.IGNORECASE)
                    characterName_tmp1 = re.sub(r'\(.+?\)',"",dmo.group(1),flags=re.I)
                    characterName_tmp = re.sub(r'\[.+?\]',"",characterName_tmp1,flags=re.I)
                    characterName = re.sub(r"^\s+|\s+$","",characterName_tmp,flags=re.I)

                    
                    #dialogue_tmp = re.sub("(\(|\[).+?\1","",dmo.group(2),flags=re.I)
                    
                    dialogue = re.sub("^\s+|\s+$","",dmo.group(2),flags=re.I)
                    
                    
                    if re.search("(\[|\()\s*(Flashback |Tag )?scene(:|;|,| )",line,re.IGNORECASE):
                        print ("Error Need to clean up data which are having 'secen:' text in middile dialogue content: -->",line)
                        sys.exit()
                    if re.match("^\s*$",characterName):
                        print ("Error ...There is no Character Name: ", line)
                        sys.exit()
                    if re.match("^\s*$",dialogue):
                        print ("Error ...There is no dialogue content: ", line)
                        sys.exit()                  
                    #print (dmo.groups())
#                elif line in keywords:
                elif re.search(keywordstr,line,re.IGNORECASE):
                    continue
                else:
                    print ("Warning ...... line is not in proper format: ", season, episode,line)
                    #sys.exit()
                    continue
                #print(season,episode,episodeName,locationName,characterName,dialogue)
                df = df.append({'Season':season, 'Episode':episode, 'Episode Name':episodeName,'Location':locationName,'Character':characterName,'Dialogue':dialogue}, ignore_index=True)

            else:
                continue
    #reak

#print (df)
df.to_csv('../dataset/friends_dataset_allseasons_tmp.csv',sep='\t',index=False)
                
                
                                                                                                                                                

In [ ]:
#Reading data from ../dataset/friends_dataset_allseasons_tmp.csv and cleaning up 'Character' and 'Location' columns
#Finally cleaned data will be stored in ../dataset/friends_dataset_allseasons.csv

import pandas as pd
import re
pd.set_option("display.max_rows", None, "display.max_columns", None)

df = pd.read_csv('../dataset/friends_dataset_allseasons_tmp.csv',sep='\t')
print(list(df.columns.values))
#print (df.head(5))

# Get the unique characters and their corresponding dialogue count
#print (df['Character'].value_counts().head(20) )

"""
#cleaning up (RACHEL|RACH|RAHCEL|Racel|Rache|Rache) in 'Character' column
df_temp = df[df['Character'].str.contains('^Ra[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
#df_temp = df[df['Character'].str.contains('^Ra', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
df['Character'] = df['Character'].replace('^\s*(RACHEL|RACH|RAHCEL|Racel|Rache|Rache)\s*$', 'Rachel', regex=True)


#cleaning up (ROSS|ross|Ross) in 'Character' column
df_temp = df[df['Character'].str.contains('^Ro[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
#df_temp = df[df['Character'].str.contains('^Ro', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
df['Character'] = df['Character'].replace('^\s*(ROSS|ross|Ross)\s*$', 'Ross', regex=True)


#cleaning up (Joey|JOEY|joey) in 'Character' column
df_temp = df[df['Character'].str.contains('^Jo[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
#df_temp = df[df['Character'].str.contains('^Jo', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
df['Character'] = df['Character'].replace('^\s*(Joey|JOEY|joey)\s*$', 'Joey', regex=True)


#cleaning up (Monica|MONICA|monica) in 'Character' column
df_temp = df[df['Character'].str.contains('^Mo[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
#df_temp = df[df['Character'].str.contains('^Mo', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
df['Character'] = df['Character'].replace('^\s*(Monica|MONICA|monica|MNCA)\s*$', 'Monica', regex=True)

#cleaning up (Phoebe|PHOEBE|Phoehe|PHOE) in 'Character' column
#df_temp = df[df['Character'].str.contains('^Ph[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
df_temp = df[df['Character'].str.contains('^Ph', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
df['Character'] = df['Character'].replace('^\s*(Phoebe|PHOEBE|Phoehe|PHOE)\s*$', 'Phoebe', regex=True)


#cleaning up (Chandler|CHANDLER|CHAN) in 'Character' column
#df_temp = df[df['Character'].str.contains('^Ch[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
df_temp = df[df['Character'].str.contains('^Ch', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
df['Character'] = df['Character'].replace('^\s*(Chandler|CHANDLER|CHAN)\s*$', 'Chandler', regex=True)



#cleaning up (Mike|MIKE|mike) in 'Character' column
#df_temp = df[df['Character'].str.contains('^Mi[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
df_temp = df[df['Character'].str.contains('^Mi', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
df['Character'] = df['Character'].replace('^\s*(Mike|MIKE|mike)\s*$', 'Mike', regex=True)

"""

"""
#cleaning up (RACHEL|RACH|RAHCEL|Racel|Rache|Rache) in 'Character' column
df_temp = df[df['Character'].str.contains('^Ra[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())

#cleaning up (ROSS|ross|Ross) in 'Character' column
df_temp = df[df['Character'].str.contains('^Ro[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())

#cleaning up (Joey|JOEY|joey) in 'Character' column
df_temp = df[df['Character'].str.contains('^Jo[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())

#cleaning up (Monica|MONICA|monica) in 'Character' column
df_temp = df[df['Character'].str.contains('^Mo[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())


#cleaning up (Phoebe|PHOEBE|Phoehe|PHOE in 'Character' column
df_temp = df[df['Character'].str.contains('^Ph[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())


#cleaning up (Phoebe|PHOEBE|Phoehe|PHOE in 'Character' column
df_temp = df[df['Character'].str.contains('^Ch[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())



#cleaning up (Phoebe|PHOEBE|Phoehe|PHOE in 'Character' column
df_temp = df[df['Character'].str.contains('^Mi[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
"""




df['Character'] = df['Character'].replace('^\s*(RACHEL|RACH|RAHCEL|Racel|Rache|Rache)\s*$', 'Rachel', regex=True)
df['Character'] = df['Character'].replace('^\s*(ROSS|ross|Ross)\s*$', 'Ross', regex=True)
df['Character'] = df['Character'].replace('^\s*(Joey|JOEY|joey)\s*$', 'Joey', regex=True)
df['Character'] = df['Character'].replace('^\s*(Monica|MONICA|monica|MNCA)\s*$', 'Monica', regex=True)
df['Character'] = df['Character'].replace('^\s*(Phoebe|PHOEBE|Phoehe|PHOE)\s*$', 'Phoebe', regex=True)
df['Character'] = df['Character'].replace('^\s*(Chandlers?|CHANDLER|CHAN)\s*$', 'Chandler', regex=True)
df['Character'] = df['Character'].replace('^\s*(Mike|MIKE|mike)\s*$', 'Mike', regex=True)
df['Character'] = df['Character'].replace('^\s*(Janice|JANICE|janice|Janice\’s Voice)\s*$', 'Janice', regex=True)
df['Character'] = df['Character'].replace('^\s*(Charlie|CHARLIE)\s*$', 'Charlie', regex=True)
df['Character'] = df['Character'].replace('^\s*(Carol|CAROL)\s*$', 'Carol', regex=True)
df['Character'] = df['Character'].replace('^\s*(Emily|EMILY)\s*$', 'Emily', regex=True)
df['Character'] = df['Character'].replace('^\s*(Mr(,|\.)?\s*Geller|Mr(,|\.)?\s*GELLER|MR(,|\.)?\s*GELLER)\s*$', 'Mr. Geller', regex=True)
df['Character'] = df['Character'].replace('^\s*(Tag|TAG)\s*$', 'Tag', regex=True)
df['Character'] = df['Character'].replace('^\s*(Paul|PAUL)\s*$', 'Paul', regex=True)
df['Character'] = df['Character'].replace('^\s*(Frank|FRANK|Frank\s+Jr\.?|FRANK\s+Jr\.?)\s*$', 'Frank Jr.', regex=True)
df['Character'] = df['Character'].replace('^\s*(Richard|RICHARD)\s*$', 'Richard', regex=True)
df['Character'] = df['Character'].replace('^\s*(Gunther|GUNTHER|GUNTER)\s*$', 'Gunther', regex=True)
df['Character'] = df['Character'].replace('^\s*(Amy|AMY)\s*$', 'Amy', regex=True)
df['Character'] = df['Character'].replace('^\s*(Mrs(,|\.)?\s*Geller|Mrs(,|\.)?\s*GELLER|MRS(,|\.)?\s*GELLER)\s*$', 'Mrs. Geller', regex=True)
df['Character'] = df['Character'].replace('^\s*(Mona|MONA)\s*$', 'Mona', regex=True)
df['Character'] = df['Character'].replace('^\s*(Woman|WOMAN)\s*$', 'Woman', regex=True)
df['Character'] = df['Character'].replace('^\s*(Pete|PETE)\s*$', 'Pete', regex=True)
df['Character'] = df['Character'].replace('^\s*(David|DAVID)\s*$', 'David', regex=True)
df['Character'] = df['Character'].replace('^\s*(Joshua|JOSHUA)\s*$', 'Joshua', regex=True)
df['Character'] = df['Character'].replace('^\s*(Gary|GARY)\s*$', 'Gary', regex=True)
df['Character'] = df['Character'].replace('^\s*(Susan|SUSAN)\s*$', 'Susan', regex=True)
df['Character'] = df['Character'].replace('^\s*(Elizabeth|ELIZABETH)\s*$', 'Elizabeth', regex=True)
df['Character'] = df['Character'].replace('^\s*(Janine|JANINE)\s*$', 'Janine', regex=True)
df['Character'] = df['Character'].replace('^\s*(Kathy|KATHY)\s*$', 'Kathy', regex=True)
df['Character'] = df['Character'].replace('^\s*(Ben|BEN)\s*$', 'Ben', regex=True)
df['Character'] = df['Character'].replace('^\s*(Jill|JILL)\s*$', 'Jill', regex=True)
df['Character'] = df['Character'].replace('^\s*(Mr(,|\.)\s*Treeger|MR(,|\.)\s*TREEGER)\s*$', 'Treeger', regex=True)
df['Character'] = df['Character'].replace('^\s*(Ursula|URSULA)\s*$', 'Ursula', regex=True)
df['Character'] = df['Character'].replace('^\s*(Eric|ERIC)\s*$', 'Eric', regex=True)
df['Character'] = df['Character'].replace('^\s*(Erica|ERICA)\s*$', 'Erica', regex=True)
df['Character'] = df['Character'].replace('^\s*(Eddie|EDDIE)\s*$', 'Eddie', regex=True)
df['Character'] = df['Character'].replace('^\s*(Gavin|GAVIN)\s*$', 'Gavin', regex=True)
df['Character'] = df['Character'].replace('^\s*(Mr\.\s*Green|MR\.\s*GREEN)\s*$', 'Green', regex=True)
df['Character'] = df['Character'].replace('^\s*(Kate|KATE)\s*$', 'Kate', regex=True)
df['Character'] = df['Character'].replace('^\s*(Danny|DANNY)\s*$', 'Danny', regex=True)
df['Character'] = df['Character'].replace('^\s*(Director|Director)\s*$', 'Director', regex=True)
df['Character'] = df['Character'].replace('^\s*(Man|MAN)\s*$', 'Man', regex=True)
df['Character'] = df['Character'].replace('^\s*(Guy|GUY)\s*$', 'Guy', regex=True)
df['Character'] = df['Character'].replace('^\s*(Nurse|NURSE)\s*$', 'Nurse', regex=True)
df['Character'] = df['Character'].replace('^\s*(Joanna|JOANNA)\s*$', 'Joanna', regex=True)
df['Character'] = df['Character'].replace('^\s*(Mark|MARK)\s*$', 'Mark', regex=True)
df['Character'] = df['Character'].replace('^\s*(Steve|STEVE)\s*$', 'Steve', regex=True)
df['Character'] = df['Character'].replace('^\s*(Will|WILL)\s*$', 'Will', regex=True)
df['Character'] = df['Character'].replace('^\s*(Doug|DOUG)\s*$', 'Doug', regex=True)
df['Character'] = df['Character'].replace('^\s*(M?r(,|\.)?\s*Zelner|M?r(,|\.)?\s*Zelner|M?R(,|\.)?\s*Zelner)\s*$', 'Mr. Zelner', regex=True)
df['Character'] = df['Character'].replace('^\s*(Mindy|MINDY)\s*$', 'Mindy', regex=True)
df['Character'] = df['Character'].replace('^\s*(Rachel\s+and\s+Monica|RACHEL\s+AND\s+MONICA|RACHEL\s+and\s+MONICA)\s*$', 'Rachel and Monica', regex=True)


#df['Location'] = df['Location'].replace("^\s*(monica and rachel\'s|monica and chandler\'s|monica|monica and rachel\'s apartment|monica and chandler\'s apartment|monica\'s apartment|monica and rachel\'s erm|chandler and monica\'s apartment|at monica and rachel\'s|monica and rachel\'s balcony|monica and phoebe\'s|chandler and monica\'s|rachel and monica\'s|? monica and chandler\'s apartment|rachel and monica\'s apartment|monica and rachels apartment|change back an forth: monica and chandler\'s apartment|monica and chandler\'s aparment|chandler and monica\'s apartement|monica and chandler\'s apartment \" continuing action|chandler arrives home and monica\'s got a video of sharks ready for chandler|monica and chandlers apartment|monica\'s apartment continued|monica and rachel|monica\'s living room|monica and rachel\'s: everyone is there and they are watching an info-mercial that stars joey|a kitchen where phoebe and monica are finishing up a catering job|monica and chandler\'s kitchen|in a tv commercial that the gang is watching at monica and rachel\'s|monica and chandler\'s apartement)\s*$", "Monica\'s Apartment", regex=True)
#df['Location'] = df['Location'].replace("^\s*(monica and rachel\'s|monica and chandler\'s|monica|monica and rachel\'s apartment|monica and chandler\'s apartment|monica\'s apartment|monica and rachel\'s erm|chandler and monica\'s apartment|at monica and rachel\'s|monica and rachel\'s balcony|monica and phoebe\'s|chandler and monica\'s|rachel and monica\'s|? monica and chandler\'s apartment|rachel and monica\'s apartment|monica and rachels apartment|change back an forth: monica and chandler\'s apartment|monica and chandler\'s aparment|chandler and monica\'s apartement)\s*$", "Monica\'s Apartment", regex=True)
#df['Location'] = df['Location'].replace("^\s*(monica and rachel\'s|monica and chandler\'s)\s*$", "Monica\'s Apartment", regex=True)
#df['Location'] = df['Location'].replace("^\s*(monica and rachel\'s|monica and chandler\'s|monica|monica and rachel\'s apartment|monica and chandler\'s apartment|monica\'s apartment|monica and rachel\'s erm|chandler and monica\'s apartment)\s*$", "Monica\'s Apartment", regex=True)

df['Location'] = df['Location'].replace("^\s*(chandler\’s and monica\’s apartment|chandler and monica\’s|monica and chandler\’s apartment|monica and rachel\’s|monica and chandler\’s|monica\’s apartment|monica and rachel\'s|monica and chandler\'s|monica|monica and rachel\'s apartment|monica and chandler\'s apartment|monica\'s apartment|monica and rachel\'s erm|chandler and monica\'s apartment|at monica and rachel\'s|monica and rachel\'s balcony|monica and phoebe\'s|chandler and monica\'s|rachel and monica\'s|\? monica and chandler\'s apartment|rachel and monica\'s apartment|monica and rachels apartment|change back an forth: monica and chandler\'s apartment|monica and chandler\'s aparment|chandler and monica\'s apartement|monica and chandler\'s apartment \" continuing action|chandler arrives home and monica\'s got a video of sharks ready for chandler|monica and chandlers apartment|monica\'s apartment continued|monica and rachel|monica\'s living room|monica and rachel\'s: everyone is there and they are watching an info-mercial that stars joey|a kitchen where phoebe and monica are finishing up a catering job|monica and chandler\'s kitchen|in a tv commercial that the gang is watching at monica and rachel\'s|monica and chandler\'s apartement|mon and chan\'s)\s*$", "Monica\'s Apartment", regex=True)
df['Location'] = df['Location'].replace("^\s*(chandler walks into joey\'s apartment|joey and janine\’s apartment|joey\'s|rachel and joey\’s|joey and now rachel\'s apartment|chandler and joey\’s|joey\’s|joey(\'|\’)s apartment|chandler joey and ross\'s|chandler joey and ross\'s apartment|joey and janine\’s|joey and rachel\'s|joey and rachel\'s apartment|chandler and joey\\'s|chandler and joey\\'s apartment|at chandler and joey\\'s|chandler\\'s office|chandler and joey\\'s erm|chandler and eddie\\'s apartment|chandler\\'s apartment|chandler and joey\\'s and ross\\'s|chandler and joey\\'s chandler is talking with his mom|back in chandler and joey\\'s apartment)\s*$", "Chandler and Joey\'s Apartment", regex=True)                                        
df['Location'] = df['Location'].replace("^\s*(ross and rachel\'s apartment|ross and rachel\’s|ross and rachel's)\s*$", "Ross and Rachel's Apartment", regex=True)                                        
df['Location'] = df['Location'].replace("^\s*(ross\'s apartment|ross\' apartment|ross\'|ross|ross\?|ross\'s room|ross\'s apartment the next morning|ross\'s place|ross\'s apartment)\s*$", "Ross's Apartment", regex=True)                                        
df['Location'] = df['Location'].replace("^\s*(phoebe\'s apartment|phoebe\’s)\s*$", "Phoebe's Apartment", regex=True)                                        
df['Location'] = df['Location'].replace("^\s*(ross is in central perk|phoebe is in central|monica enters central perk|centr?al perk|inside central perk|central perk\’s)\s*$", "Central Perk", regex=True)                                        




# Get the unique characters and their corresponding dialogue count
print (df['Character'].value_counts().head(20) )
print (df['Location'].value_counts().head(20) )

"""
#cleaning up 'Character' column
df_temp = df[df['Character'].str.contains('^Em', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
df['Character'] = df['Character'].replace('^\s*(Emily|EMILY)\s*$', 'Emily', regex=True)

df_temp = df[df['Character'].str.contains('^Em[^ /]+\s*$', flags=re.IGNORECASE, regex=True)]
print (df_temp['Character'].value_counts())
"""

#print (df)
df.to_csv('../dataset/friends_dataset_allseasons.csv',sep='\t',index=False)


Initial findings are below, need to work on Analysis 

Total number of Lines of Lead Characters:

Rachel        9294
Ross          9175
Chandler      8454
Monica        8437
Joey          8198
Phoebe        7531

Total number of Lines of Top 5 supporting character:

Mike            360
Richard         254
Mr. Geller      231
Janice          219
Carol           205

Top 5 sets:

Monica's Apartment               18561
Central Perk                     10896
Chandler and Joey's Apartment     7188
Ross's Apartment                  1228
the hallway                        692
